# Predefined Sequences

Instead of uploading waveforms defined as *numpy* arrays, the `zhinst-toolkit` also provides pre-defined sequences for standard experiments. These differ between UHFQA and HDAWG and include:
 
 * Rabi (HDAWG)
 * T1 (HDAWG)
 * T2 Ramsey (HDAWG)
 * Trigger (HDAWG)
 * Continuous-Wave (CW) Spectroscopy (UHFQA)
 * Pulsed Spectroscopy (UHFQA)
 * Multiplexed Readout (UHFQA)

Here we show some basic examples of predefined sequences and how they can be used with the *HDAWG* and *UHFQA* together in a real-life experiment. To ensure that the two devices use the same Data Server, we use the *zhinst-toolkit's* `MultiDeviceConnection` and to setup and connect a Data Server and then connect the two devices to it.

In [3]:
import zhinst.toolkit as tk
import numpy as np

# create a 'Multi Device Connection'
mdc = tk.MultiDeviceConnection(host="10.42.0.226")
mdc.setup()

# connect the devices
mdc.connect_device(tk.HDAWG("hdawg 1", "dev8030"))
mdc.connect_device(tk.UHFQA("uhfqa 1", "dev2266"))

# references to the instruments are held in attributes 'hdawgs' and 'uhfqas'
hdawg = mdc.hdawgs["hdawg 1"]
uhfqa = mdc.uhfqas["uhfqa 1"]

## Resonator Spectroscopy

For a resonator spectroscopy we can use the predefined `"Pulsed Spectroscopy"` sequence on the UHFQA. It plays a rectangular waveform of duration `pulse_length`. We configure one AWG Core of the *HDAWG* to a `"Trigger"` sequence to trigger the spectroscopy pulses on the *UHFQA* (`trigger_mode="External Trigger"`). 

In [7]:
# assign AWG Cores
trigger = hdawg.awgs[0]
readout = uhfqa.awg

# configure trigger AWG
trigger.set_sequence_params(
    sequence_type="Trigger",
    period=10e-6,
    repetitions=1e3,
)
trigger.compile()

# configure UHFQA to Pulsed Spectroscopy
readout.set_sequence_params(
    sequence_type="Pulsed Spectroscopy",
    period=10e-6,
    pulse_lenth=2e-6,
    repetitions=1e3,
    trigger_mode= "External Trigger",   # UHFQA triggered by Master Trigger
)
readout.compile()

Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded


The spectroscopy sequence enables output modulation on the *UHFQA*. The signals on the two output channels are modulated with the sine and cosine of the internal oscillator. Its frequency is set by the parameter `uhfqa.nodetree.osc.freq`. On the signal acquisition side, the *integration mode: "Spectroscopy"* is used, which demodulates the signal input with the same internal oscillator.

A sweep of the IF modulation frequency could be done like this: 

In [ ]:
# configure results to length 1000 and 1 hardware averages
uhfqa.result_source("Integration")
uhfqa.nodetree.qa.result.length(1e3)
uhfqa.nodetree.qa.result.averages(1)
modulation_freq = uhfqa.nodetree.osc.freq

# define the sweep points and an empty array for results
frequencies = np.linspace(50e6, 100e6, 101)
results = np.array([])

for f in frequencies:
    modulation_freq(f)                               # set modulation frequency
    readout.run()                                    # start readout AWG
    trigger.run()                                    # start trigger AWG
    trigger.wait_done()                              # wait until trigger AWG has finished
    avg_result = np.mean(uhfqa.channels[0].result()) # average the result vector
    results = np.append(results, avg_result)         # append to results

## Qubit Spectroscopy

A drive pulse for qubit spectroscopy can be implemented in one of two ways. Either the predefined `"Rabi"` sequence (see below) is used with a single amplitude point and a long pulse width. The option would be to use the `"Simple"` sequence and upload the drive pulse of choice as a *numpy array*. For simplicity we will show the latter option and play a simle rectangular pulse of *5 us* duration on the *HDAWG*.

On the *UHFQA*, we use the predefined `"Readout"` sequence. This sequence creates a readout tone for every enabled *Readout Channel* of the *UHFQA* at their respective `readout_frequency`. By enabling the readout channel we also program the integration weights of the readout channel to demodulate the signal at the given `readout_frequency`. 

In [8]:
# assign AWGs
drive = hdawg.awgs[0]
readout = uhfqa.awg
channel = uhfqa.channels[0]

# configure trigger AWG
drive.set_sequence_params(
    sequence_type="Simple",
    period=10e-6,
    repetitions=1e3,
)
drive.reset_queue()
wave = np.ones(int(2.4e9 * 5e-6))  # sampling rate x pulse duration
drive.queue_waveform(wave, wave)
drive.compile_and_upload_waveforms()

# configure UHFQA to Readout
channel.enable()
channel.readout_frequency(123e6)
readout.set_sequence_params(
    sequence_type="Readout",
    period=10e-6,
    pulse_lenth=2e-6,
    repetitions=1e3,
    trigger_mode= "External Trigger",   # UHFQA triggered by Master Trigger
)
readout.compile()

Current length of queue: 1
Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.071 s
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded


To sweep the frequency of the drive signal, we want to enable modulation of the ouput signal of the qubit drive at with the internal oscillator. This is done by enabling the IQ Modulation mode of the AWG Core (modulation with the internal oscillator for IQ signal generation). The frequency of the oscillator can then be swept over the desired range.

In [ ]:
frequencies = np.linspace(20e6, 50e6, 101)
results = np.array([])

# enable IQ Modulation mode
drive.enable_iq_modulation()

for f in frequencies:
    drive.modulation_freq(f)
    readout.run()
    drive.run()
    drive.wait_done()
    avg_result = np.mean(channel.result())
    results = np.append(results, avg_result)


## Rabi Sequence

The *Rabi Sequence* plays a simple Gaussian pulse that is defined by the sequence parameters `pulse_width` and `pulse_truncation`. The parameter `pulse_amplitudes` expects a *list* or *array* of values for the amplitude sweep. With the IQ Modulation mode enabled a typical Rabi sequence would look like this:

In [17]:
# assign AWGs
drive = hdawg.awgs[0]
readout = uhfqa.awg
channel = uhfqa.channels[0]

# define numpy array with Rabi Amplutdes
rabi_amplitudes = np.linspace(0, 1.0, 101)

# configure Rabi Amplitude Sweep
hdawg.awgs[0].enable_iq_modulation()
hdawg.awgs[0].modulation_freq(100e6)

hdawg.awgs[0].set_sequence_params(
    sequence_type="Rabi",
    pulse_width=50e-9,
    pulse_amplitudes=rabi_amplitudes,
)
hdawg.awgs[0].compile()

# configure UHFQA to Readout
uhfqa.nodetree.qa.result.length(1e3 * len(rabi_amplitudes))
uhfqa.nodetree.qa.result.averages(1)

channel.enable()
channel.readout_frequency(123e6)
readout.set_sequence_params(
    sequence_type="Readout",
    period=10e-6,
    pulse_lenth=2e-6,
    repetitions=1e3 * len(rabi_amplitudes),  # play averages x sweep-points readout pulses
    trigger_mode= "External Trigger",        # UHFQA triggered by Master Trigger
)
readout.compile()

Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded


In [ ]:
readout.run()
drive.run()
drive.wait_done()
result = channel.result()

## Time Domain Characterization: T1 & T2*

Time domain characterization can be done with predefined sequences *T1* and *T2 Ramsey*. They work in the same way as the *Rabi Sequence*. However, instead of passing an array of sweep values to the parameter `pulse_amplitudes`, the parameter is called `time_delays`. This parameter defines the  

In [17]:
# assign AWGs
drive = hdawg.awgs[0]
readout = uhfqa.awg
channel = uhfqa.channels[0]

# define numpy array with Delay Times
delay_times = np.linspace(0.5e-6, 20e-6, 40)

# configure Rabi Amplitude Sweep
hdawg.awgs[0].enable_iq_modulation()
hdawg.awgs[0].modulation_freq(100e6)

hdawg.awgs[0].set_sequence_params(
    sequence_type="T1",
    pulse_width=50e-9,
    delay_times=delay_times,
)
hdawg.awgs[0].compile()

# configure UHFQA to Readout
uhfqa.nodetree.qa.result.length(1e3 * len(delay_times))
uhfqa.nodetree.qa.result.averages(1)

channel.enable()
channel.readout_frequency(123e6)
readout.set_sequence_params(
    sequence_type="Readout",
    period=10e-6,
    pulse_lenth=2e-6,
    repetitions=1e3 * len(delay_times),  # play averages x sweep-points readout pulses
    trigger_mode= "External Trigger",        # UHFQA triggered by Master Trigger
)
readout.compile()

Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded


In [ ]:
readout.run()
drive.run()
drive.wait_done()
result = channel.result()

## Multiplexed Readout

```
              HDAWG 1
           +-----------+                    ______________________________
+----<----+   AWG 1   |  Trigger          _|                              |_________:_
|         +-----------+                    :                              :         :
+----+----->  AWG 2   |  AWGs[0]          _:________________________|XXXXX|_________:_
     |    |-----------+                    :                              :         :
     +----->  AWG 3   |  AWGs[1]          _:_____________________|XXXXXXXX|_________:_
     |    |-----------+                    :                              :         :
     +----->  AWG 4   |  AWGs[2]          _:__________________|XXXXXXXXXXX|_________:_
     |    +-----------+                    :                              :         :
     |                                     :                              :         :
     |     UHFQA                           :                              :         :
     |    +-----------+                    :                              :         :
     +----->          | Readout           _:______________________________|XXXXX|___:_
          +-----------+
```